# Praktikum 4 - Klasifikasi dengan ANN

**Nama : Nanda Putra Khamdani**

**Kelas : TI-3H / 18**

**NIM : 2241720180**

### Deskripsi

Pada praktikum kali ini, Anda diminta untuk membuat model ANN untuk mengklasifikasi potensi seorang customer akan meninggalkan perusahaan Anda atau tidak. Istirlah populer dari fenomena ini disebut sebagai 'churn'. Tingkat churn yang tinggi (chrun rate) akan berdampak tidak baik bagi perusahaan.

## Perhatian!

Pada praktikum ini, Anda akan menggunakan library tensorflow dari google. Oleh karena itu, Anda diharuskan untuk menginstal tensorflow terlebih dahulu.

Anda juga perlu menyesuaikan instalasi tensorflow yang Anda gunakan pada komputer lokal, apakah komputasi pada,

  * CPU

  * GPU (GPU support CUDA)

  * Apple Silicon (M1/M2)



## Pra Pengolahan Data

* Langkah 1 - Import Library

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

* Langkah 2 - Load Data

In [3]:
dataset = pd.read_csv('../Jobsheet9_Perceptron-ANN/Data/Churn_Modelling.csv')
print(dataset.head())
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         790

Kode di atas memuat dataset Churn Modelling dari file CSV menggunakan pandas dan menampilkan lima baris pertama dari dataset dengan print(dataset.head()) untuk memberikan gambaran awal tentang data yang dimiliki. Setelah itu, variabel X diisi dengan nilai fitur dari dataset, yaitu semua kolom kecuali kolom pertama, kedua, dan kolom terakhir, menggunakan metode iloc. Sementara itu, variabel y berisi label target, yaitu nilai dari kolom terakhir dalam dataset. Dengan demikian, kode ini mempersiapkan data untuk analisis atau pelatihan model dengan memisahkan fitur dan label target.

---

In [4]:
print(f"X: ", X)
print(f"y: ", y)

X:  [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y:  [1 0 1 ... 1 1 0]


* Langkah 3 - Encoding Data Kategorikal

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

Kode di atas melakukan proses encoding untuk kolom ketiga dalam variabel X, yang berisi data kategorikal, menggunakan LabelEncoder dari scikit-learn. Pertama, objek LabelEncoder diinisialisasi, dan kemudian kolom ketiga dari array X diubah dengan menerapkan metode fit_transform, yang mengonversi nilai kategorikal menjadi angka unik.

---

In [6]:
print(f"X: ", X)

X:  [[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


* Langkah 4 - Encoding Kolom "Geography" dengan One Hot Encoder

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

Kode di atas menggunakan ColumnTransformer dari scikit-learn untuk menerapkan OneHotEncoder pada kolom kedua dari variabel X, yang berisi data kategorikal. Dengan menginisialisasi objek ColumnTransformer, kode ini menetapkan OneHotEncoder sebagai transformasi untuk kolom kedua, sedangkan kolom lainnya tetap tidak berubah berkat parameter remainder='passthrough'. Metode fit_transform kemudian diterapkan untuk mengubah data, menghasilkan array X yang telah dikonversi menjadi format numerik dengan representasi one-hot untuk kolom yang dikategorikan, sehingga memungkinkan model pembelajaran mesin untuk memproses fitur secara efektif. 

---

In [8]:
print(f"X: ", X)

X:  [[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


* Langkah 5 - Split Data

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

* Langkah 6 - Scaling Fitur

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Membuat Model ANN

* Langkah 1 - Inisiasi Model ANN

In [11]:
ann = tf.keras.models.Sequential()

Kode di atas menginisialisasi sebuah model Artificial Neural Network (ANN) menggunakan Keras, yang merupakan bagian dari TensorFlow. Dengan memanggil tf.keras.models.Sequential(), sebuah objek model bertipe Sequential dibuat. Model ini memungkinkan penambahan lapisan-lapisan (layers) secara berurutan, di mana output dari satu lapisan menjadi input untuk lapisan berikutnya. 

---

* Langkah 2 - Membuat Input Layer dan Hidden Layer Pertama

In [12]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

Kode di atas menambahkan lapisan Dense ke dalam model Artificial Neural Network (ANN) yang telah dibuat sebelumnya menggunakan Keras. Dengan memanggil ann.add(tf.keras.layers.Dense(units=6, activation='relu')), sebuah lapisan tersembunyi baru ditambahkan ke model dengan 6 unit neuron dan menggunakan fungsi aktivasi ReLU (Rectified Linear Unit).

Fungsi aktivasi ReLU populer digunakan dalam jaringan saraf karena kemampuannya untuk mengatasi masalah vanishing gradient, yang sering terjadi pada fungsi aktivasi lainnya. Dalam konteks ini, setiap neuron di lapisan ini akan mengeluarkan nilai positif dari inputnya atau nol jika inputnya negatif, sehingga memperkenalkan non-linearitas ke dalam model.

---

* Langkah 3 - Membuat Hidden Layer Kedua

In [13]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

* Langkah 4 - Membuat Output Layer

In [14]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

Kode di atas menambahkan lapisan keluaran (output layer) ke dalam model Artificial Neural Network (ANN) yang telah dibangun menggunakan Keras. Dengan memanggil ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid')), sebuah lapisan Dense ditambahkan dengan 1 unit neuron dan menggunakan fungsi aktivasi sigmoid.

Fungsi aktivasi sigmoid menghasilkan nilai antara 0 dan 1, yang membuatnya cocok untuk digunakan pada model klasifikasi biner. Dalam konteks ini, lapisan keluaran ini akan memberikan prediksi probabilitas untuk kelas target, di mana nilai mendekati 1 menunjukkan kemungkinan yang tinggi untuk salah satu kelas, sedangkan nilai mendekati 0 menunjukkan kemungkinan yang rendah. 

---

## Training Model

* Langkah 1 - Compile Model (Menyatukan Arsitektur) ANN

In [15]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Kode di atas mengonfigurasi model Artificial Neural Network (ANN) dengan menggunakan metode compile. Parameter yang ditetapkan mencakup penggunaan optimizer Adam, yang terkenal efisien dalam memperbarui bobot model selama proses pelatihan, serta fungsi kehilangan binary crossentropy, yang cocok untuk klasifikasi biner, membantu menghitung perbedaan antara prediksi model dan nilai target. Selain itu, akurasi ditetapkan sebagai metrik untuk mengevaluasi kinerja model. Dengan pengaturan ini, model siap untuk dilatih menggunakan metode fit, memanfaatkan optimizer, fungsi kehilangan, dan metrik yang telah ditentukan untuk meningkatkan kemampuannya dalam melakukan klasifikasi.

---

* Langkah 2 - Fitting Model

In [16]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 885us/step - accuracy: 0.7886 - loss: 0.5389
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 928us/step - accuracy: 0.7957 - loss: 0.4730
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 726us/step - accuracy: 0.7990 - loss: 0.4471
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 741us/step - accuracy: 0.8003 - loss: 0.4343
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 697us/step - accuracy: 0.7965 - loss: 0.4309
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step - accuracy: 0.7882 - loss: 0.4354
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 771us/step - accuracy: 0.8090 - loss: 0.4318
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 892us/step - accuracy: 0.8220 - loss: 0.4193
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 621us/step - accuracy: 0.8250 - loss: 0.4212
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 827us/step - accuracy: 0.8318 - loss: 0.4179
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 620us/step - accuracy: 0.8307 - loss: 0.4183
Epoch 12/100
250/25

Kode di atas menjalankan proses pelatihan untuk model Artificial Neural Network (ANN) yang telah dikonfigurasi sebelumnya dengan menggunakan metode fit. Dalam pemanggilan ann.fit(X_train, y_train, batch_size=32, epochs=100), dataset fitur X_train dan label target y_train digunakan sebagai input. Parameter batch_size=32 mengatur jumlah sampel yang akan diproses sebelum memperbarui bobot model, sementara epochs=100 menunjukkan bahwa model akan dilatih selama 100 iterasi penuh melalui dataset. 

---

## Membuat Prediksi

Diberikan informasi sebagai berikut,

  * Geography: France

  * Credit Score: 600

  * Gender: Male

  * Age: 40 years old

  * Tenure: 3 years

  * Balance: $ 60000

  * Number of Products: 2

  * Does this customer have a credit card ? Yes

  * Is this customer an Active Member: Yes

  * Estimated Salary: $ 50000

Apakah customer tersebut perlu dipertahankan?

## Modelkan Data Baru dan Buat Prediksi



In [17]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
[[False]]


* Apakah hasilnya False?

    Iya hasilnya false, Hasil prediksi model ANN yang menunjukkan nilai False berarti bahwa pelanggan dengan karakteristik yang diberikan tidak perlu dipertahankan. Meskipun pelanggan tersebut memiliki saldo tinggi sebesar $60,000 dan dua produk, beberapa faktor lain berkontribusi pada keputusan ini. Skor kredit sebesar 600 dapat mengindikasikan risiko kredit yang lebih tinggi, sementara masa keanggotaan selama 3 tahun dianggap relatif singkat, menunjukkan kurangnya loyalitas. Selain itu, usia 40 tahun dan gaji $50,000 mungkin tidak cukup untuk menunjukkan stabilitas finansial. Model ANN, yang dilatih berdasarkan data historis, mungkin telah mengenali pola bahwa pelanggan dengan karakteristik serupa lebih cenderung untuk churn. Dengan threshold prediksi yang ditetapkan pada 0.5, nilai prediksi di bawah batas ini menunjukkan bahwa pelanggan tersebut diperkirakan tidak akan bertahan dengan perusahaan.

---

## Prediksi Dengan Data Testing

In [18]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 645us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


Dalam kode di atas, model ANN digunakan untuk memprediksi potensi churn pelanggan pada data pengujian X_test. Pertama, prediksi dilakukan dengan memanggil metode predict() pada model ANN, yang menghasilkan nilai probabilitas untuk setiap sampel dalam X_test. Selanjutnya, nilai probabilitas tersebut diubah menjadi label biner dengan membandingkannya terhadap threshold 0.5; jika nilai probabilitas lebih besar dari 0.5, maka dianggap sebagai kelas 1 (pelanggan yang mungkin akan churn), dan jika tidak, dianggap sebagai kelas 0 (pelanggan yang tidak mungkin churn).

Kemudian, hasil prediksi (y_pred) dan label asli dari data pengujian (y_test) digabungkan dalam bentuk array dua kolom menggunakan np.concatenate(). Hasilnya adalah matriks yang menunjukkan label prediksi dari model bersamaan dengan label aktual, yang memungkinkan analisis lebih lanjut tentang akurasi dan performa model dalam mengklasifikasikan churn pelanggan. 

---

## Cek Akurasi dan Confusion Matrix

In [47]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1514   81]
 [ 191  214]]


0.864

Hasil evaluasi model Artificial Neural Network (ANN) untuk memprediksi churn pelanggan menunjukkan performa yang cukup baik. Confusion matrix yang dihasilkan adalah [[1514, 81], [191, 214]], di mana model berhasil mengidentifikasi 214 pelanggan yang benar-benar churn (True Positives) dan 1514 pelanggan yang tidak churn (True Negatives). Namun, terdapat 81 pelanggan yang salah diprediksi sebagai churn (False Positives) dan 191 pelanggan yang sebenarnya churn tetapi tidak teridentifikasi oleh model (False Negatives). Akurasi keseluruhan model mencapai 86.4%, yang menunjukkan bahwa model berhasil mengklasifikasikan sekitar 86.4% dari total data pengujian dengan benar.

---